In [1]:
import pickle
import pyarrow.parquet as pq
import pandas as pd
pickled_model = pickle.load(open('final_model_jaccard_2.pkl', 'rb'))
# pickled_model.prepare()

In [2]:
import tqdm

In [15]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier


def get_scores(X, y, x_predict, return_names=False, return_only_names=False):
    if return_only_names is True and return_names is False:
        raise ValueError("return_only_names cannot be True if return_names is False")
    clf = LogisticRegression(max_iter=3000).fit(X.to_numpy(), y)
    scores = clf.predict_proba([x_predict])
    if return_names is False:
        return scores
    else:
        classes = clf.classes_
        scores = scores.tolist()[0]
        classes = [(classes[i], scores[i]) for i in range(10)]
        classes.sort(key=lambda x: x[1], reverse=True)
        if return_only_names is True:
            classes = [i[0] for i in classes]
            # print('результаты', classes)
        return classes


def get_scores_catboost(X, y, x_predict, return_names=False, return_only_names=False):
    if return_only_names is True and return_names is False:
        raise ValueError("return_only_names cannot be True if return_names is False")
    clf = CatBoostClassifier(iterations=1000, learning_rate=0.01, logging_level='Silent').fit(X, y)
    scores = clf.predict_proba([x_predict])
    if return_names is False:
        return scores
    else:
        classes = clf.classes_
        scores = scores.tolist()[0]
        classes = [(classes[i], scores[i]) for i in range(10)]
        classes.sort(key=lambda x: x[1], reverse=True)
        if return_only_names is True:
            classes = [i[0] for i in classes]
            # print(classes)
        return classes

In [38]:
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
from pynndescent import NNDescent
from sklearn import preprocessing
from tqdm import tqdm


def vid_long(videos: DataFrame, user_id: int, users: DataFrame) -> DataFrame:
    tqdm.pandas()
    hist = users[users["user_id"] == user_id]
    hist = pd.merge(hist, videos, on='item_id').drop(columns=["user_id"], axis=1)
    for i in tqdm(range(100)):
        hist[f"v_title_{i}"] = hist.progress_apply(lambda row: row[f"v_title_{i}"] * 2
        if ((row["watch_time"] / (row["duration"] / 1000)) > 0.25
            if (row["duration"] / 1000) > 300
            else row["watch_time"] > 30)
        else 1, axis=1)
        hist[f"v_title_{i}"] = preprocessing.minmax_scale(hist[f"v_title_{i}"].T).T
    return hist.drop(columns=["watch_time", "duration"], axis=1)


def like(videos: pd.DataFrame, user_id: int, emotions: pd.DataFrame) -> pd.DataFrame:
    emotions = emotions[["user_id", "item_id", "type"]]
    emotions = emotions[emotions["user_id"] == user_id]
    emotions = emotions[["item_id", "type"]]
    # hist = users[users["user_id"] == user_id]
    # hist = pd.merge(hist, videos, on='item_id').drop(columns=["user_id"], axis=1)
    # hist = pd.merge(hist, emotions, left_on='item_id', right_on="C3")
    tqdm.pandas()
    new_videos = videos.merge(emotions, on="item_id")
    if len(new_videos) != 0:
        videos = new_videos
        for i in tqdm(range(100)):
            videos[f"vid_title_{i}"] = videos.progress_apply(
                    lambda row: row[f"vid_title_{i}"] * 2 if row["type"] == "pos_emotions"
                    else (0.5 if row["type"] == "neg_emotions" else 1), axis=1).to_numpy()
            videos[f"vid_title_{i}"] = preprocessing.minmax_scale(np.array([videos[f"vid_title_{i}"]]).T).T
            videos[f"vid_descr_{i}"] = preprocessing.minmax_scale(np.array([videos.progress_apply(
                    lambda row: row[f"vid_descr_{i}"] * 2 if row["type"] == "pos_emotions"
                    else (0.5 if row["type"] == "neg_emotions" else 1), axis=1).to_numpy()]).T).T

            # normalize(np.array([videos.progress_apply(
            #         lambda row: row[f"vtitle{i}"] * 2 if row["C4"] == "pos_emotions"
            #         else (0.5 if row["C4"] == "neg_emotions" else 1), axis=1).to_numpy()]))[0]
        return videos.drop(columns=["type"], axis=1)
    else:
        return videos
    # return videos.drop(columns=["type"], axis=1)


def make_vector(data: DataFrame, video_ids: list[int], users: DataFrame, emotions: DataFrame, user_id:str):
    # print("история просмотра", video_ids)
    df = data[data["item_id"].isin(video_ids)]

    # vid_long(data, user_id, users)
    # likes = like(df, user_id, emotions)

    # TODO add weights
    return np.average(df.drop(['item_id', 'video_description', 'video_title', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1).values, axis=0)
    # return np.mean(likes.drop(['item_id', 'video_description', 'video_title', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1).values, axis=0)


def get_video_corpus(data: DataFrame, index: NNDescent, vector: np.ndarray):
    df = data.drop(['item_id', 'video_description', 'video_title', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1).columns.to_list()

    top100nearest = index.query(pd.DataFrame([vector], columns=df), k=10)
    vids_indices = top100nearest[0][0]
    res_data = data.iloc[vids_indices]

    return res_data.drop(['item_id', 'video_description', 'video_title', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1), res_data['item_id'].values


def get_10_category(train_hist: pd.DataFrame, new_hist: pd.DataFrame = None) -> list:
    if new_hist is not None:
        train_hist = train_hist.append(new_hist, ignore_index=True)
    cat_columns = [col for col in tqdm(train_hist.columns) if col.startswith('cat')]
    cat_data = train_hist[cat_columns]

    # подсчет количества 1 в каждом столбце
    counts = cat_data.sum()

    # сортировка столбцов по убыванию количества 1 и вывод первых 10
    top_columns = counts.sort_values(ascending=False)[:10]
    return top_columns


def get_top_videos_in_cat(cat_name, train_hist: pd.DataFrame, new_hist: pd.DataFrame = None) -> str:
    if new_hist is not None:
        train_hist = train_hist.append(new_hist, ignore_index=True)
    cat_data = train_hist.loc[train_hist[cat_name] == 1]
    counts = cat_data['item_id'].value_counts()
    return counts.index[0]


def get_top_videos(train_hist: pd.DataFrame, new_hist: pd.DataFrame = None) -> list:
    top_category = get_10_category(train_hist, new_hist)
    return [get_top_videos_in_cat(i, train_hist, new_hist) for i in tqdm(top_category)]


def get_popular() -> np.ndarray:
    popular = pd.read_csv("top_cat.csv")
    popular = data["item_id"].to_numpy()
    popular = np.reshape(popular, (10, 10))
    result = []
    for i in range(10):
        result.append(popular[i][random.randint(0, 9)])
    result = np.random.permutation(result)
    return result


def create_submission_file(path: str, data: DataFrame, index, users: DataFrame, emotions: DataFrame):
    test_file = pd.read_csv(path)
    user_ids = test_file["user_id"].values
    preds = []
    for idx, user_id in enumerate(tqdm(user_ids)):
        x_predict = make_vector(data, users[users["user_id"] == user_id]["item_id"].values, users, emotions, user_id)
        if len(x_predict) == 0:
            preds.append(get_popular)
            break
        corpus, target = get_video_corpus(data, index, x_predict)
        preds.append(target)
        # preds.append(get_scores(corpus, target, x_predict, return_names=True, return_only_names=True))

        if idx % 5 == 0:
            submission = pd.DataFrame({"user_id": user_ids[0:idx], "recs": preds})
            submission.to_csv("submission" + idx + ".csv", index=False)


    submission = pd.DataFrame({"user_id": user_ids, "recs": preds})
    submission.to_csv("submission.csv", index=False)


In [3]:
import pandas as pd
import numpy as np
from pynndescent import NNDescent

# pickled_model = pickle.load(open('final_model_jaccard.pkl', 'rb'))
# print(0)
# pickled_model.prepare()
# data = pd.read_csv("final_df.csv")
# print('1')
users = pq.read_table('train_dataset_RUTUBE/player_starts_train.parquet').to_pandas()
print('2')
# train_hist = pd.merge(users, data, on='item_id').drop(columns=["user_id"], axis=1)
# try:
    # new_users = pd.read_csv("new_player_starts_train.csv")
# except FileNotFoundError:
#     new_users = None
# new_hist = pd.merge(new_users, data, on='item_id').drop(columns=["user_id"], axis=1)
print('3')
emotions = pd.read_csv("train_dataset_RUTUBE/emotions.csv")

0


KeyboardInterrupt: 

In [7]:
data = pd.read_csv("final_df_2.csv")

/var/folders/vn/wf1rpzp51jj_bp2lz71rs3q80000gn/T/ipykernel_784/257456476.py:1: DtypeWarning: Columns (252) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("final_df_2.csv")


In [10]:
data.drop('Unnamed: 0', inplace=True, axis=1)

In [6]:
users = pq.read_table('train_dataset_RUTUBE/player_starts_train.parquet').to_pandas()

In [5]:
emotions = pd.read_csv("train_dataset_RUTUBE/emotions.csv")

In [21]:
data['ctr.CTR_10days_21_07'].fillna(data['ctr.CTR_10days_21_07'].mode()[0], inplace=True)
data['ctr.CTR_10days_01_08'].fillna(data['ctr.CTR_10days_01_08'].mode()[0], inplace=True)
data['ctr.CTR_10days_10_08'].fillna(data['ctr.CTR_10days_10_08'].mode()[0], inplace=True)
data['ctr.CTR_10days_21_08'].fillna(data['ctr.CTR_10days_21_08'].mode()[0], inplace=True)

In [24]:
data.drop(['tv_title'], axis=1, inplace=True)

In [ ]:
data[data.isna().any(axis=1)]

In [26]:
print(data.head())

         item_id  vid_title_0  vid_title_1  vid_title_2  vid_title_3  \
0   video_165654     0.317030     0.583350     0.319950     0.233488   
1  video_1173704     0.481264     0.387798     0.602036     0.802317   
2    video_23927     0.648496     0.440843     0.160715     0.613938   
3  video_1003780     0.203357     0.307920     0.590847     0.516022   
4   video_105383     0.744297     0.835887     0.621679     0.773973   

   vid_title_4  vid_title_5  vid_title_6  vid_title_7  vid_title_8  ...  \
0     0.397311     0.335484     0.683385     0.544738     0.459705  ...   
1     0.655285     0.626316     0.562532     0.271270     0.515960  ...   
2     0.367600     0.542256     0.304734     0.719216     0.541566  ...   
3     0.755484     0.283917     0.717133     0.625334     0.525521  ...   
4     0.504829     0.539626     0.493591     0.560223     0.590561  ...   

   ctr.CTR_10days_01_08  ctr.CTR_10days_10_08  ctr.CTR_10days_21_08  \
0                   0.0                   0.0

In [8]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
# neighbors = pickled_model.query(test.drop(['item_id', 'video_description', 'tv_title', 'video_title', 'CTR_10days_01_08', 'CTR_10days_21_07', 'CTR_10days_10_08', 'CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1).head(10), k=100)

In [ ]:
# pickled_model.query(train.drop(['item_id', 'video_description', 'tv_title', 'video_title', 'CTR_10days_01_08', 'CTR_10days_21_07', 'CTR_10days_10_08', 'CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1).head(10), k=100)

In [ ]:
# for i in neighbors[0][1]:
#     print(data[data['item_id'] == train.iloc[i].item_id].video_title)

In [11]:
# for video_id in test.head(10)['item_id']:
#     print(data[data['item_id'] == video_id].video_title)

1025987    Арай Чобанян показал палату в которой рожает И...
Name: video_title, dtype: object
468859    LAUNCH ● X-DIAG PRO3 и DIAGZONE PRO ● АКТИВАЦИ...
Name: video_title, dtype: object
1638170    1992  Реклама Барби Маттел Голливудские Волосы...
Name: video_title, dtype: object
423766    АСМР Карта Нурсултана в Казахстане ??
Name: video_title, dtype: object
1597720    Micro Machines (NES Dendy 8bit) - Full Walkthr...
Name: video_title, dtype: object
1698657    Кинетический песок АСМР • Видео для расслаблен...
Name: video_title, dtype: object
67347    Зефирантес Выскочка_Расцвел Зефирантес_Красота...
Name: video_title, dtype: object
872393    «Повышенная государственная академическая стип...
Name: video_title, dtype: object
1564649    Вестница весны - под музыку, для любителей про...
Name: video_title, dtype: object
79079    А. Брицын "Рождество" И. Дунаевский "Песня о в...
Name: video_title, dtype: object


In [39]:
create_submission_file("train_dataset_RUTUBE/sample_submission.csv", data, pickled_model, users, emotions)

  0%|          | 0/97240 [00:39<?, ?it/s]


ValueError: All arrays must be of the same length

In [ ]:
data.mean()

In [35]:
print(data.dropna())

               item_id  vid_title_0  vid_title_1  vid_title_2  vid_title_3  \
0         video_165654     0.317030     0.583350     0.319950     0.233488   
1        video_1173704     0.481264     0.387798     0.602036     0.802317   
2          video_23927     0.648496     0.440843     0.160715     0.613938   
3        video_1003780     0.203357     0.307920     0.590847     0.516022   
4         video_105383     0.744297     0.835887     0.621679     0.773973   
...                ...          ...          ...          ...          ...   
1808057  video_1018741     0.670306     0.513039     0.521495     0.845573   
1808058    video_37551     0.620435     0.581103     0.651037     0.274357   
1808059   video_575652     0.333735     0.495146     0.391018     0.275300   
1808060   video_763853     0.500728     0.563369     0.658009     0.390734   
1808061  video_1973653     0.378764     0.417246     0.445300     0.238808   

         vid_title_4  vid_title_5  vid_title_6  vid_title_7  vi

In [100]:
print(data[data['item_id'] == 'user_25219604'].video_title)

Series([], Name: video_title, dtype: object)


In [117]:
v = ['video_1759835', 'video_1805778', 'video_1850494', 'video_1008588', 'video_1755974', 'video_1406947', 'video_1050337', 'video_1302615', 'video_1575452', 'video_1654596']


for i in v:
    print(data[data['item_id'] == i].video_title)

1501290    Налоговые новости от Аркадия Брызгалина (9.02.23)
Name: video_title, dtype: object
781754    HilalDeep - Darkness (Original Mix)
Name: video_title, dtype: object
1418745    Глазовское водохранилище Владимирская область июль 2018
Name: video_title, dtype: object
1476713    Слепить сиреноголового из пластилина / Как победить сиреноголового и откуда он взялся
Name: video_title, dtype: object
737086    Катасонов, Левченко, Хазин анализируют состояние ЦБ РФ, экономики РФ дают прогнозы развития ситу...
Name: video_title, dtype: object
29344    История появления мультяшного кота scp/ Картун кэт из пластилина
Name: video_title, dtype: object
1275446    ТЕСТ - i7 12700k мать GIGABYTE Z690 GAMING X DDR4 DDR4-3600mh GTX 1050ti BATTLEFIELD 4
Name: video_title, dtype: object
370710    Новый мост Гонконг - Чжухай - Макао: как доехать из Гонконга на автобусе. Отзыв Ездили Знаем
Name: video_title, dtype: object
597171    VID_20230819_104842_544
Name: video_title, dtype: object
939760    Стр

In [130]:
user_videos = ['video_1905049', 'video_222807', 'video_2087988', 'video_1516664',
 'video_1516664']

In [135]:
for v in user_videos:
    print(data[data['item_id'] == v].video_description)

1409603    Гости программы «Детали» — психолог Татьяна Руденко, мама первоклассницы Евгения Феоктистова.\n\nЛетние каникулы — лучшая часть учебного года, для детей уж точно. Они отдыхают, не думают о контрольных и экзаменах. Но настрой пора менять. Новый учебный год не за горами. Чтобы он прошел гладко, важно настроить ребенка на рабочую волну. Как это сделать, рассказали в программе "Детали".
Name: video_description, dtype: object
212104    «Четыре свадьбы» отправляются в Турцию! Невесты из Стамбула Анастасия и Ирина покажут, как проходят национальные турецкие свадьбы. Анастасия позовёт на своё мероприятие 700 гостей, а Ирина устроит «ночь хны». Свадьбы Ольги и Олеси пройдут в Аланье. Ольга выберет для своего торжества стиль бохо, а Олеся – романтический стиль. Понравятся ли девушкам мероприятия соперниц, узнаем из программы «Четыре свадьбы».
Name: video_description, dtype: object
313824    В специальном выпуске программы «Четыре свадьбы» мы вспомним яркие моменты мероприятий 112 неве

In [115]:
recs = ['video_1135690', 'video_1174633', 'video_1084905', 'video_1123959', 'video_1193351', 'video_1250023', 'video_1277946', 'video_1369958', 'video_142279', 'video_15449']

In [134]:
pd.options.display.max_colwidth = 1000

for i in recs:
    print(data[data['item_id'] == i].video_description)

1058118    Музыкальный центр, Aiwa NSX, музыкальные тесты\n\nФильм состоит из нескольких частей. Инструкция, распаковка, прослушивание, тесты и ремонт музыкального центра Aiwa NSX V900, сравнение с версией AV90. Глава по ремонту может быть полезной для людей, интересующихся этими аппаратами и желающими привести их в работоспособное состояние. Когда я начинал этим заниматься, примерно полгода назад, то смотрел видеоролики на эту тему в разных источниках. Некоторые моменты были за кадром, и приходилось додумывать самому. Постараюсь в этих фильмах показать все этапы восстановления центра практически без вырезания.
Name: video_description, dtype: object
335799    Показываю как играть на гитаре песню Король и Шут - Кукла колдуна. Обозначаю аккорды и табы. Так же сделала кавер с эффектом караоке.\n\nVK: http://vk.com/corusmusic\nInstagram: skripa\n\n#КорольиШут #КакИграть #Аккорды #КуклаКолдуна #гитара #разбор
Name: video_description, dtype: object
1705623    • Премьера: ►11 августа 2022◄\n•

In [138]:
data.columns[101]

'vid_descr_0'